In [ ]:
import math

# Constants
WHEEL_BASE = 1400.0
WHEEL_RADIUS = 250.0
WHEEL_CLEARANCE_TARGET = 25.4
TUBE_HEIGHT = 152.4
TUBE_OFFSET = TUBE_HEIGHT / 2.0
BUCKET_FRONT_CLEARANCE = 1.5 * 25.4 # 38.1
ARM_PIVOT_Y = 200.0
FRAME_Z_OFFSET = 150.0
ARM_PIVOT_Z_HEIGHT = 950.0

# Calculated Constants
ARM_PIVOT_Z = FRAME_Z_OFFSET + ARM_PIVOT_Z_HEIGHT # 1100
R_CONSTRAINT = WHEEL_RADIUS + WHEEL_CLEARANCE_TARGET + TUBE_OFFSET # 351.6

# Coordinates
P = (ARM_PIVOT_Y, ARM_PIVOT_Z) # (200, 1100)
# C = (WHEEL_BASE, FRAME_Z_OFFSET + WHEEL_RADIUS) # (1400, 400) note: using user's Z=400
C = (1400.0, 400.0)

# Option B: Corrected formula (Clearance from Front of Wheel)
T_B_x = WHEEL_BASE + WHEEL_RADIUS + BUCKET_FRONT_CLEARANCE
T_B_z = 73.0
T_B = (T_B_x, T_B_z)

def solve_geometry(Tip):
    vec_PC = (C[0]-P[0], C[1]-P[1])
    dist_PC = math.sqrt(vec_PC[0]**2 + vec_PC[1]**2)
    angle_PC = math.atan2(vec_PC[1], vec_PC[0])
    
    alpha_PC = math.asin(R_CONSTRAINT / dist_PC)
    angle_tangent_main = angle_PC + alpha_PC
    u_main = (math.cos(angle_tangent_main), math.sin(angle_tangent_main))
    
    vec_TC = (C[0]-Tip[0], C[1]-Tip[1])
    dist_TC = math.sqrt(vec_TC[0]**2 + vec_TC[1]**2)
    angle_TC = math.atan2(vec_TC[1], vec_TC[0])
    
    if dist_TC < R_CONSTRAINT:
        print("ERROR: Tip T is inside clearance zone! Cannot solve tangent.")
        return

    alpha_TC = math.asin(R_CONSTRAINT / dist_TC)
    angle_tangent_drop = angle_TC - alpha_TC
    u_drop = (math.cos(angle_tangent_drop), math.sin(angle_tangent_drop))
    
    det = u_main[0] * (-u_drop[1]) - u_main[1] * (-u_drop[0])
    
    dx = Tip[0] - P[0]
    dy = Tip[1] - P[1]
    
    det_s = dx * (-u_drop[1]) - dy * (-u_drop[0])
    s = det_s / det
    E = (P[0] + s * u_main[0], P[1] + s * u_main[1])
    
    vec_PE = (E[0]-P[0], E[1]-P[1])
    vec_ET = (Tip[0]-E[0], Tip[1]-E[1])
    
    mag_PE = math.sqrt(vec_PE[0]**2 + vec_PE[1]**2)
    mag_ET = math.sqrt(vec_ET[0]**2 + vec_ET[1]**2)
    
    dot = vec_PE[0]*vec_ET[0] + vec_PE[1]*vec_ET[1]
    val = dot / (mag_PE * mag_ET)
    val = max(-1.0, min(1.0, val))
    
    angle_rad = math.acos(val)
    angle_deg = math.degrees(angle_rad)
    
    print(f"E=({E[0]:.2f}, {E[1]:.2f})")
    print(f"L1 (Main)={mag_PE:.2f}")
    print(f"L2 (Drop)={mag_ET:.2f}")
    print(f"Angle_Dev={angle_deg:.2f}")
    print(f"SHAPE_ANGLE={180 - angle_deg:.2f}")

solve_geometry(T_B)